In [1]:
import os
import pandas as pd
import numpy as np
import time
import turicreate

In [2]:
t_start = time.time()

In [3]:
#Buat directory submission dan model
try:
    os.mkdir('submission')
except:
    pass
try:
    os.mkdir('model')
except:
    pass

# Load Dataset

In [4]:
#Dataset
cd_train = pd.read_csv('dataset/coupon_detail_train.csv')
cl_test = pd.read_csv('dataset/coupon_list_test.csv')
cl_train = pd.read_csv('dataset/coupon_list_train.csv')
cv_train = pd.read_csv('dataset/coupon_visit_train.csv')
user_list = pd.read_csv('dataset/user_list.csv')

# Preprocessing

In [5]:
#list kupon test yang pernah divisit user

visited_test = cv_train[['USER_ID_hash','VIEW_COUPON_ID_hash']].merge(cl_test['COUPON_ID_hash'], how='inner', left_on='VIEW_COUPON_ID_hash', right_on='COUPON_ID_hash').drop('VIEW_COUPON_ID_hash', axis=1)
print(len(visited_test))
visited_test.head(5)

741


,USER_ID_hash,COUPON_ID_hash
0,118d41a61656bbfe22ee73294f45289c,7d7487370022eb18ee130856aa1c6eec
1,7afc4e216672636f61dfbad40d9816c6,7d7487370022eb18ee130856aa1c6eec
2,32f93b1872c1fe4e7e590d0305a3f02d,7d7487370022eb18ee130856aa1c6eec
3,cd017554ad7e16899df7b004a7321863,7d7487370022eb18ee130856aa1c6eec
4,cd017554ad7e16899df7b004a7321863,7d7487370022eb18ee130856aa1c6eec


In [6]:
#list kupon yang pernah dibeli user + total pembelian

user_coupon_pcount = cd_train.groupby(['USER_ID_hash', 'COUPON_ID_hash'])['PURCHASEID_hash'].size().reset_index(name='PURCHASE_COUNT')
print(len(user_coupon_pcount))
user_coupon_pcount.head(5)

158933


,USER_ID_hash,COUPON_ID_hash,PURCHASE_COUNT
0,0000b53e182165208887ba65c079fc21,38beeadfe3f97e640367eddae4a8c1b5,1
1,00035b86e6884589ec8d28fbf2fe7757,25a27d420caa1c46a8d3c0572d27868a,1
2,0005b1068d5f2b8f2a7c978fcfe1ca06,4a79cd05ecb2bf8672e1d955f5faa7fa,1
3,0005b1068d5f2b8f2a7c978fcfe1ca06,f0f66195d527a5a9509e139ed367b879,1
4,000cc06982785a19e2a2fdb40b1c9d59,229ff5cc21c8d26615493be7f3b42841,1


In [7]:
#Drop fitur yang tidak terpakai

cl_train.drop(['DISPFROM','DISPEND','VALIDFROM','VALIDEND'],axis=1,inplace=True)
cl_test.drop(['DISPFROM','DISPEND','VALIDFROM','VALIDEND'],axis=1,inplace=True)

In [8]:
#Fillna VALIDPERIOD, USABLE_DATE_n

cl_train.fillna(-1, inplace=True)
cl_test.fillna(-1, inplace=True)

In [9]:
#Fungsi untuk quantize VALIDPERIOD dan DISPPERIOD dalam:
#-1 : Null
#0 : Dibawah seminggu
#1 : 1-2 minggu
#2 : 2-4 minggu
#3 : di atas sebulan
def quantizePeriod(period):
    if period == -1:
        return period
    elif period <=7:
        return 0
    elif period <=14:
        return 1
    elif period <= 30:
        return 2
    else:
        return 3

In [10]:
cl_train['DISPPERIOD'] = cl_train.apply(lambda x: quantizePeriod(x.DISPPERIOD), axis=1)
cl_test['DISPPERIOD'] = cl_test.apply(lambda x: quantizePeriod(x.DISPPERIOD), axis=1)
cl_train['VALIDPERIOD'] = cl_train.apply(lambda x: quantizePeriod(x.VALIDPERIOD), axis=1)
cl_test['VALIDPERIOD'] = cl_test.apply(lambda x: quantizePeriod(x.VALIDPERIOD), axis=1)

cl_train['DISPPERIOD'] = cl_train['DISPPERIOD'].astype('str')
cl_test['DISPPERIOD'] = cl_test['DISPPERIOD'].astype('str')
cl_train['VALIDPERIOD'] = cl_train['VALIDPERIOD'].astype('str')
cl_test['VALIDPERIOD'] = cl_test['VALIDPERIOD'].astype('str')

In [11]:
#Normalisasi harga dengan log function
cl_train['CATALOG_PRICE'] = np.log(1+cl_train['CATALOG_PRICE'])
cl_train['DISCOUNT_PRICE'] = np.log(1+cl_train['DISCOUNT_PRICE'])

cl_test['CATALOG_PRICE'] = np.log(1+cl_test['CATALOG_PRICE'])
cl_test['DISCOUNT_PRICE'] = np.log(1+cl_test['DISCOUNT_PRICE'])

In [12]:
#Ubah Fitur USABLE_DATE_n menjadi categorical
UD_features = ['USABLE_DATE_MON', 'USABLE_DATE_TUE','USABLE_DATE_WED','USABLE_DATE_THU', 'USABLE_DATE_FRI','USABLE_DATE_SAT','USABLE_DATE_SUN','USABLE_DATE_HOLIDAY','USABLE_DATE_BEFORE_HOLIDAY']

cl_train[UD_features] = cl_train[UD_features].astype('str')
cl_test[UD_features] = cl_test[UD_features].astype('str')

# Convert Data to SFrame

In [13]:
#Ubah kedalam SFrame
#on .create
observation_data = turicreate.SFrame(user_coupon_pcount)

item_data = turicreate.SFrame(pd.concat([cl_train,cl_test]))

In [14]:
#Ubah kedalam SFrame
#on .recommend
users = turicreate.SFrame(user_list[['USER_ID_hash']])

items = turicreate.SFrame(cl_test[['COUPON_ID_hash']])

# Fitting

In [15]:
model = turicreate.recommender.item_content_recommender.create(item_data=item_data, observation_data=observation_data, item_id='COUPON_ID_hash', user_id='USER_ID_hash', target='PURCHASE_COUNT', similarity_metrics='auto', item_data_transform='auto', verbose=True)
model.save('model/Turi_ItemContentRec_Model.mdl') #Bentuknya folder

#Load Model : turicreate.load_model('model/Turi_ItemContentRec_Model.mdl')

Applying transform:
Class             : AutoVectorizer

Model Fields
------------
Features          : ['CAPSULE_TEXT', 'GENRE_NAME', 'PRICE_RATE', 'CATALOG_PRICE', 'DISCOUNT_PRICE', 'DISPPERIOD', 'VALIDPERIOD', 'USABLE_DATE_MON', 'USABLE_DATE_TUE', 'USABLE_DATE_WED', 'USABLE_DATE_THU', 'USABLE_DATE_FRI', 'USABLE_DATE_SAT', 'USABLE_DATE_SUN', 'USABLE_DATE_HOLIDAY', 'USABLE_DATE_BEFORE_HOLIDAY', 'large_area_name', 'ken_name', 'small_area_name']
Excluded Features : ['COUPON_ID_hash']

Column                      Type   Interpretation  Transforms  Output Type
--------------------------  -----  --------------  ----------  -----------
CAPSULE_TEXT                str    categorical     None        str        
GENRE_NAME                  str    categorical     None        str        
PRICE_RATE                  int    numerical       None        int        
CATALOG_PRICE               float  numerical       None        float      
DISCOUNT_PRICE              float  numerical       None        

Starting brute force nearest neighbors model training.

Validating distance components.

Initializing model data.

Initializing distances.

...

Data has 158933 observations with 22782 users and 19723 items.

Data prepared in: 0.401101s

Loading user-provided nearest items.

Generating candidate set for working with new users.

Finished training in 0.036395s

# Recommend

In [16]:
#Recommend dengan k sebanyak jumlah kupon yang tersedia
res = model.recommend(users, k=len(cl_test), items=items).to_dataframe()

recommendations finished on 1000/22873 queries. users per second: 48683.1

recommendations finished on 2000/22873 queries. users per second: 48579.1

recommendations finished on 3000/22873 queries. users per second: 47005.7

recommendations finished on 4000/22873 queries. users per second: 46531.6

recommendations finished on 5000/22873 queries. users per second: 45115.8

...

recommendations finished on 18000/22873 queries. users per second: 30605.6

recommendations finished on 19000/22873 queries. users per second: 31184.6

recommendations finished on 20000/22873 queries. users per second: 31661

recommendations finished on 21000/22873 queries. users per second: 32114.6

recommendations finished on 22000/22873 queries. users per second: 31713.2

In [17]:
# Beri bonus skor untuk kupon yang pernah divisit user.

visited_test['visit_bonus'] = 10000
res = res.merge(visited_test, on=['USER_ID_hash', 'COUPON_ID_hash'], how='left')

res.visit_bonus.fillna(0, inplace=True)
res.score.fillna(res.score.min(), inplace=True)

In [18]:
# Sort skor kembali

res['final_score'] = res['score'] + res['visit_bonus']

res.sort_values(by='final_score', ascending=False, inplace=True)

# Convert to Submission Format

In [19]:
def clean_prediction(row):
    data = row.PURCHASED_COUPONS[:10]
    data = str("".join(str(data))[2:-2].replace("', '"," "))
    return data

In [20]:
res = res.groupby('USER_ID_hash')['COUPON_ID_hash'].apply(list).reset_index(name='PURCHASED_COUPONS')

In [21]:
res['PURCHASED_COUPONS'] = res.apply(clean_prediction, axis=1)

In [22]:
sub_name = 'submission/sub_CPP_turi_ItemContentRec_'+str(int(time.time()))+'.csv'

res.to_csv(sub_name, index=False)

In [23]:
res.head(10)

,USER_ID_hash,PURCHASED_COUPONS
0,0000b53e182165208887ba65c079fc21,16a9c255c41dbeb9c8484dfc782a2a89 c76ea297ebd3a...
1,00035b86e6884589ec8d28fbf2fe7757,f5a77f2907876411752d58e1b9030023 7792efbb9eb86...
2,0005b1068d5f2b8f2a7c978fcfe1ca06,0c015306597566b632bebfb63b7e59f3 b0a17a6d92b78...
3,000cc06982785a19e2a2fdb40b1c9d59,c988d799bc7db9254fe865ee6cf2d4ff ea370226d85d0...
4,0013518e41c416cd6a181d277dd8ca0b,c988d799bc7db9254fe865ee6cf2d4ff 63eeb16b672d0...
5,001acdee812a18acfd7509172bed5700,f25c0927423d87a8403f474c2f09f339 df5155df3dc03...
6,001fd7876e3aa29393537c6baf308e43,cb4c67c749dc53afd24aea31de705108 6940e1d50cb76...
7,002383753c1e5d6305c8aff6f89e26d6,691ac7cbfd8359eb01c93cf44da1416d dc23d9b9f9cf7...
8,0025cae7997d25ea5cf8851bb099c798,e0e86297922c5799ee19881ca999b81d d1da92a020c07...
9,002822059a01d895fad84f2f2ff5c1f1,e4db7645ae556f252e60636df7c8eac8 79de77aa8c36f...


## Submit to Kaggle

In [24]:
#Submit ke kaggle, opsional
try:
    import kaggle
    !kaggle competitions submit -c coupon-purchase-prediction -f $sub_name -m $sub_name

    found = False
    time_out_counter = time.time()

    while 1:
        submission_list = !kaggle competitions submissions -c coupon-purchase-prediction
        for sub_row in submission_list:
            if sub_name in sub_row and 'complete' in sub_row:
                scores = sub_row[sub_row.find('complete')+10:].split()
                print('\n\n',sub_name,'\nPrivate : ',scores[1],'\t|\tPublic : ',scores[0], sep='')
                found = True
                break
            if (time.time()-time_out_counter) > 180:
                print('\n\nTime Limit Exceeded (3 min)\nPlease check again later on :\nhttps://www.kaggle.com/c/coupon-purchase-prediction/submissions')
                found = True
                break
        if found:
            break
        time.sleep(3)
except:
    print('Kaggle API not found')

100%|███████████████████████████████████████| 7.92M/7.92M [00:35<00:00, 233kB/s]
Successfully submitted to Coupon Purchase Prediction

submission/sub_CPP_turi_ItemContentRec_1591012590.csv
Private : 0.00636	|	Public : 0.00855


In [25]:
print('Duration (H:M:S):',time.strftime('%H:%M:%S', time.gmtime(time.time()-t_start)))

Duration (H:M:S): 00:01:46
